In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [25]:
df_google = pd.read_csv('../data/Google-Playstore.csv')
df_apple = pd.read_csv('../data/appleAppData.csv')

In [3]:
print("------------Google------------------")
df_google.info()
print("Shape = " + str(df_google.shape))
print("------------Apple------------------")
df_apple.info()
print("Shape = " + str(df_apple.shape))

------------Google------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), f

In [27]:
print("Number of null values in Android dataset")
df_google.isnull().sum()


Number of null values in Android dataset


App Name                  2
App Id                    0
Category                  0
Rating                22883
Rating Count          22883
Installs                107
Minimum Installs        107
Maximum Installs          0
Free                      0
Price                     0
Currency                135
Size                    196
Minimum Android        6530
Developer Id             33
Developer Website    760835
Developer Email          31
Released              71053
Last Updated              0
Content Rating            0
Privacy Policy       420953
Ad Supported              0
In App Purchases          0
Editors Choice            0
Scraped Time              0
dtype: int64

In [28]:
print("Number of null values in Apple dataset")
df_apple.isnull().sum()

Number of null values in Apple dataset


App_Id                          0
App_Name                        1
AppStore_Url                    0
Primary_Genre                   0
Content_Rating                  0
Size_Bytes                    224
Required_IOS_Version            0
Released                        3
Updated                         0
Version                         0
Price                         490
Currency                        0
Free                            0
DeveloperId                     0
Developer                       0
Developer_Url                1109
Developer_Website          643988
Average_User_Rating             0
Reviews                         0
Current_Version_Score           0
Current_Version_Reviews         0
dtype: int64

In [4]:
def replace_value(df,cmp,val,des):
        print(des + str(len(df.loc[df[cmp].isnull()])))
        df.loc[df[cmp].isnull(),cmp] = val

In [5]:
# Drop rows without app name is null
no_name = df_google[df_google['App Name'].isnull()].index
print("[google] apps without name " + str(len(no_name)))
df_google.drop(no_name,inplace=True)


no_name = df_apple[df_apple['App_Name'].isnull()].index
print("[apple] apps without name " + str(len(no_name)))
df_apple.drop(no_name,inplace=True)

# rename google "App Name" into "App_Name"
df_apple.rename(columns={"App_Name": "App Name"},inplace=True)


[google] apps without name 2
[apple] apps without name 1


In [6]:
# Replace null rating with 0

replace_value(df_google,"Rating",0,"[google] rating null ")

# rename Apple "Average_User_Rating" into Rating
df_apple.rename(columns={"Average_User_Rating": "Rating"},inplace=True)
replace_value(df_apple,"Rating",0,"[apple] rating null ")



[google] rating null 22883
[apple] rating null 0


In [7]:
# Replace null price with 0

replace_value(df_google,"Price",0,"[google] price null ")
replace_value(df_apple,"Price",0,"[apple] price null ")


[google] price null 0
[apple] price null 490


In [8]:
# Currency, Convert all google other currency into xxx
print("[google] other currency count " + str(len( df_google.loc[df_google["Currency"] != "USD"] )))
df_google.loc[df_google["Currency"] != "USD","Currency"] = "XXX"

[google] other currency count 1396


In [9]:
# Fixing the "rating count" column


replace_value(df_google,"Rating Count",0,"[google] Rating Count null ")
# Convert Rating count into int
df_google["Rating Count"].astype('int')

# Replace apple column Reviews into Rating Count
df_apple.rename(columns={"Reviews":"Rating Count"},inplace=True)
replace_value(df_apple,"Rating Count",0,"[apple] Rating Count null ")


[google] Rating Count null 22883
[apple] Rating Count null 0


In [10]:
# Fixing Installs on android by taking the average of "Minimum Installs" and "Maximum Installs"
# No install column for apple

df_google["Installs"] = round((df_google["Minimum Installs"] + df_google["Maximum Installs"]) / 2)

In [11]:
# Rename App_Id to App Id

df_apple.rename(columns={"App_Id":"App Id"},inplace=True)

In [12]:
# replace apple Size_Bytes column name
df_apple.rename(columns={"Size_Bytes":"Size"},inplace=True)

replace_value(df_google,"Size",0,"[google] Size null ")
replace_value(df_apple,"Size",0,"[apple] Size null ")


# Convert values to bytes
def toByte(d):
    d = str(d)
    d = d.replace(",",'');
    l = d.strip()[-1].lower()
    
    if l == 'm':
        return int(float(d[0:-1]))*1024*1024;
    if l == "g":
        return int(float(d[0:-1]))*1024*1024*1024;
    if l == "k":
        return int(float(d[0:-1]))*1024;
    if l == '0':
        return 0;
    if d == 'Varies with device':
        return 0;
    return int(d);
    
df_google["Size"] = df_google["Size"].apply(toByte)

m = df_google["Size"].mean()


# replace google and apple value "Varies with device" and null with mean
print("[google] size mean " + str(m))
df_google.loc[df_google["Size"] == 0,"Size"] = m

m = df_apple["Size"].mean()

print("[apple] size mean " + str(m))
df_apple.loc[df_apple["Size"] == 0,"Size"] = m



[google] Size null 196
[apple] Size null 224
[google] size mean 19266901.24980566
[apple] size mean 75094942.08582748


In [13]:
# "Minimum Android" remove string and comvert to float

df_google["Minimum Android"] = df_google["Minimum Android"].replace("and up","",regex=True).str.strip()

# replace Null with the lowset value 
lst = df_google["Minimum Android"].unique().astype(str)
lst.sort()

print("[google] Minimum Android in the dataset is = " + lst[0])
replace_value(df_google,"Minimum Android",lst[0],"[google] Minimum Android null ")

# rename apple and android to Minimum OS
df_google.rename(columns={"Minimum Android":"Minimum OS"},inplace=True)
df_apple.rename(columns={"Required_IOS_Version":"Minimum OS"},inplace=True)

[google] Minimum Android in the dataset is = 1.0
[google] Minimum Android null 6530


In [14]:
# Filling the values of Android released column with the nearest value
print("[google] Released null " + str(len( df_google.loc[df_google["Released"].isnull()] )))
df_google["Released"].fillna(method='bfill',inplace=True)
df_google["Released"] = pd.to_datetime(df_google["Released"],format="%b %d, %Y")

df_apple["Released"] = pd.to_datetime(df_apple["Released"],format="%Y-%m-%d")

[google] Released null 71053


In [15]:
# rename android column from "Last Updated" to "Updated"

df_google.rename(columns={"Last Updated": "Updated"},inplace=True)
df_google["Released"] = pd.to_datetime(df_google["Released"],format="%b %d, %Y")
df_google["Updated"] = pd.to_datetime(df_google["Updated"],format="%b %d, %Y")

In [16]:
# rename apple column "Content_Rating" into "Content Rating"

df_apple.rename(columns={"Content_Rating":"Content Rating"},inplace=True)

In [17]:
# rename android column "Developer Id" into Developer

df_google.rename(columns={"Developer Id": "Developer"},inplace=True)

In [18]:
# rename apple Developer_Website into Developer Website
df_apple.rename(columns={"Developer_Website":"Developer Website"},inplace=True)

In [19]:
# replace empty google "Developer Website" with email and remove the gmail from the email address
df_google.loc[df_google["Developer Website"].isnull(),"Developer Website"] = df_google["Developer Email"].str.replace(".gmail.com","",regex=True)

In [20]:
# rename apple Primary_Genre into Category
df_apple.rename(columns={"Primary_Genre":"Category"},inplace=True)

lst_android = [
    {'Adventure':"Games"},
    {'Tools':"Utilities"},
    {'Productivity':""},
    {'Communication':"Social"},
    {'Social':""},
    {'Libraries & Demo':"Books"},
    {'Lifestyle':""},
    {'Personalization':"Utilities"},
    {'Racing':"Games"},
    {'Maps & Navigation':"Navigation"},
    {'Travel & Local':"Travel"},
    {'Food & Drink':""},
    {'Books & Reference':"Reference"},
    {'Medical':""},
    {'Puzzle':"Games"},
    {'Entertainment':""},
    {'Arcade':"Games"},
    {'Auto & Vehicles':"Sports"},
    {'Photography':"Photo & Video"},
    {'Health & Fitness':""},
    {'Education':""},
    {'Shopping':""},
    {'Board':"Games"},
    {'Music & Audio':"Music"},
    {'Sports':""},
    {'Beauty':"Health & Fitness"},
    {'Business':""},
    {'Educational':"Games"},
    {'Finance':""},
    {'News & Magazines':"Magazines & Newspapers"},
    {'Casual':"Games"},
    {'Art & Design':"Graphics & Design"},
    {'House & Home':"Lifestyle"},
    {'Card':"Games"},
    {'Events':"Lifestyle"},
    {'Trivia':"Games"},
    {'Weather':""},
    {'Strategy':"Games"},
    {'Word':"Games"},
    {'Video Players & Editors':"Photo & Video"},
    {'Action':"Games"},
    {'Simulation':"Games"},
    {'Music':""},
    {'Dating':"Social"},
    {'Role Playing':"Games"},
    {'Casino':"Games"},
    {'Comics':"Book"},
    {'Parenting':"Book"}
]


lst_apple = [
    {'Education':""},
    {'Book':"Books"},
    {'Reference':""},
    {'News':""},
    {'Lifestyle':""},
    {'Health & Fitness':""},
    {'Games':""},
    {'Medical':""},
    {'Food & Drink':""},
    {'Sports':""},
    {'Shopping':""},
    {'Entertainment':""},
    {'Utilities':""},
    {'Stickers':"Utilities"},
    {'Productivity':""},
    {'Music':""},
    {'Photo & Video':""},
    {'Travel':""},
    {'Business':""},
    {'Social Networking':"Social"},
    {'Magazines & Newspapers':""},
    {'Navigation':""},
    {'Finance':""},
    {'Weather':""},
    {'Graphics & Design':""},
    {'Developer Tools':"Utilities"}
]





In [21]:
# Map apple and google categories to each other
def replace_categories(df, cats): 
    lst = [i for i in cats if i[list(i.keys())[0]] != ""]
    for i in lst:
        df.loc[df["Category"] == list(i.keys())[0],"Category"] = i[list(i.keys())[0]]
    
replace_categories(df_google,lst_android)
replace_categories(df_apple,lst_apple)

In [22]:
# Removing unnecessary columns

df_google.drop(columns=["Minimum Installs",
                        "Maximum Installs","Developer Email","Privacy Policy","Scraped Time"],inplace=True)

df_apple.drop(columns=["AppStore_Url","Version",
                       "DeveloperId","Developer_Url",
                       "Current_Version_Score","Current_Version_Reviews"],inplace=True)

In [23]:
df_google.to_csv("../data/google_cleaned.csv")
df_apple.to_csv("../data/apple_cleaned.csv")

In [24]:
print("Done")

Done
